# NOTES


- SBU est 7 jam 

- RUN ON: tanggal 5 jam 15 -> tanggal 5 jam 8

- Corn Config: 0 0 8 5 1/1 ? *



## this notebook was part 1/2 of SBU level


# i. Config

In [ ]:

import numpy as np
import pandas as pd
import json
import re
import time
from urllib.parse import quote
import sqlalchemy as sql
from sqlalchemy.pool import NullPool
import gc
from datetime import datetime, timedelta
from sqlalchemy.sql import text
import calendar
import pickle

In [ ]:
gc.collect()

In [ ]:
# Connection to DWH
username = ""
password = "********"
host = "" 


connect_string = 'mysql://'+username+':'+quote(password)+'@'+host+'/?charset=utf8mb4'
sql_engine_server = sql.create_engine(connect_string, poolclass=NullPool)
sql_engine_server.connect()

# configure max lenght
query = text("SET GLOBAL group_concat_max_len = 10000")
sql_engine_server.execute(query)

In [ ]:

#test Conection
connect_string


In [ ]:

#test concection
sql.create_engine(connect_string)

## ii. Declare Function

In [ ]:

#get transaction period for the current update fot the threshold period

def monthdelta(date, delta):
    m, y = (date.month+delta) % 12, date.year + ((date.month)+delta-1) //12
    if not m: m = 12
    d = min(date.day, 
        [31, 29 if y%4==0 and (not y%100==0 or y%400 == 0) else 28,
        31,30,31,30,31,31,30,31,30,31][m-1])
    return date.replace(day=d,month=m,year=y)
    
m=monthdelta(datetime.today(), -1).month
y=monthdelta(datetime.today(), -1).year
d=calendar.monthrange(y, m)[1]
prev_year = datetime(y, m, d).year-1
end_period = str(datetime(y, m, d).date())


In [ ]:
end_period

In [ ]:
# generate SBU from helper

q1="""SELECT DISTINCT SBU FROM gclub.rfm_helper"""
df_sbu0 = pd.read_sql_query(q1, sql_engine_server)
df_sbu0

In [ ]:
# additional if some SBU want to exclude

# df_sbu_0 = df_sbu0.iloc[:5]
# df_sbu_1 = df_sbu0.iloc[6:]
# df_sbu = pd.concat([df_sbu_0,df_sbu_1])
# df_sbu= df_sbu0.iloc[5:6]

df_sbu = df_sbu0

In [ ]:
# mappping


all_sbu_mapping = []

for sbu_list in df_sbu['SBU']:
    all_sbu_mapping.append(
    {

    f'{sbu_list}': 
    {
        
    'helper_rfm':
    f""" SELECT
        rule.TransactionID,
        rule.MemberID,
        rule.AtLocationTime,
        results.RetailValueAfterTax VALUE,
        store.ConceptName
    FROM mardiyan.LoyaltyRuleServiceTransactions PARTITION(P2023,P2022,P2021)  rule
    INNER JOIN mardiyan.LoyaltyRuleServiceTransactionResults PARTITION(P2023,P2022,P2021)  results
        on rule.TransactionID = results.TransactionID
    LEFT join
    mardiyan.MasterStoresV2 store
    ON
        rule.LocationReference = store.StoreCode
        AND rule.AtLocationTime BETWEEN store.ValidFrom AND store.ValidUntil    
    LEFT JOIN gclub.rfm_helper help 
    ON
    store.conceptname = help.`Concept Name`
    WHERE
        help.SBU =  "{sbu_list}"
        AND rule.AtLocationTime > Date_sub('<end_period>', INTERVAL 2 year)
        AND rule.AtLocationTime <= '<end_period>'
        AND rule.Status = 'COMPLETED'
        """,

    'rfm_query':
    f"""SELECT
    b.MemberID AS member_id,
    "{sbu_list}" AS SBU,
    'dynamic' AS threshold ,
    Date_sub('<end_period>', INTERVAL 2 YEAR) AS start_period,
    '<end_period>' AS end_period,
    COUNT(DISTINCT(b.TransactionID)) freq,
    SUM(b.VALUE) total_value,
    AVG(b.VALUE) avg_value,
    max(DATEDIFF(b.AtLocationTime, '<end_period>')) recency_days
    FROM
    dev_db.temp_rfm_market b
    GROUP BY
    b.MemberID
    """,
    
    'helper_forgot':
    f"""SELECT
            DISTINCT a.ssoid
        FROM
            gclub.MemberIDgroupBySales_2016_2018 a 
            LEFT JOIN gclub.rfm_helper help 
            ON
            a.ConceptName = help.`Concept Name`
        WHERE
            help.SBU = "{sbu_list}"
    UNION ALL
        select DISTINCT rule.memberid ssoid  
        FROM
        mardiyan.LoyaltyRuleServiceTransactions PARTITION(P2023,P2022,P2021,P2020,P2019) rule
    LEFT JOIN mardiyan.MasterStoresV2 store ON
            rule.LocationReference = store.StoreCode
            AND rule.AtLocationTime BETWEEN store.ValidFrom AND store.ValidUntil
    LEFT JOIN gclub.rfm_helper help 
    ON
    store.conceptname = help.`Concept Name`
    WHERE
        help.SBU ="{sbu_list}"
            AND rule.atlocationtime >= '2019-01-01'
            AND rule.atlocationtime <= '<end_period>' 
            AND rule.Status = 'COMPLETED' """ ,
    
    'forgotten_user_query': 
    f"""
   SELECT
    "{sbu_list}" SBU,
    Date_sub('<end_period>', INTERVAL 2 YEAR) start_period,
    '<end_period>' end_period,
    (
    SELECT
        COUNT(DISTINCT dev_db.temp_rfm_market_forgotten_users.ssoid)
    FROM
         dev_db.temp_rfm_market_forgotten_users ) - <count_current_rfm> forgotten_users
         """
}
}
    )

In [ ]:

len(all_sbu_mapping)

In [ ]:

_error_concept = []

In [ ]:

class rfm:
    
    def __init__(self, df, segment_map, threshold={}, order='rfm'):
        self.segment_map=segment_map
        self.order=order
        self.df = df
        self.threshold = threshold.copy()
        #if the threshold is not set manually, it will be automatically set using the df distribution (quantile)
        
        if self.threshold=={}:
            self.threshold['recency'] = [self.df.recency_days.quantile(.25),self.df.recency_days.quantile(.5),self.df.recency_days.quantile(.75)]
            self.threshold['frequency'] = [self.df.freq.quantile(.25),self.df.freq.quantile(.5),self.df.freq.quantile(.75)]
            self.threshold['monetary'] = [self.df.total_value.quantile(.25),self.df.total_value.quantile(.5),self.df.total_value.quantile(.75)]
            
    #set the threshold manually
    def set_threshold(self,threshold={}):
        self.threshold = threshold
        

    def tier_marker(self,series,metric):
        q = self.threshold[metric]
        return series.apply(lambda x: 4 if x <= q[0] else 3 if (x > q[0] and x <= q[1]) else 2 if (x > q[1] and x <= q[2]) else 1  )
        
        
    def fit(self):
        self.df['recency_tier'] = self. tier_marker(self.df.recency_days,'recency')
        self.df['freq_tier'] = self.tier_marker(self.df.freq,'frequency')
        self.df['total_value_tier'] = self.tier_marker(self.df.total_value,'monetary')
        #self.df['avg_value_tier'] = self.tier_marker(self.df.avg_value)

        if self.order=='mfr':
            self.df['combined_tier'] = self.df.apply(lambda x :   str(x.total_value_tier)+ str(x.freq_tier) + str(x.recency_tier), axis=1 )
        elif self.order=='fmr':
            self.df['combined_tier'] = self.df.apply(lambda x :   str(x.freq_tier) + str(x.total_value_tier)+str(x.recency_tier), axis=1 )
        elif self.order=='frm':
            self.df['combined_tier'] = self.df.apply(lambda x :   str(x.freq_tier) +str(x.recency_tier) + str(x.total_value_tier), axis=1 )
        else:
            #rfm order
            self.df['combined_tier'] = self.df.apply(lambda x : str(x.recency_tier) + str(x.freq_tier) + str(x.total_value_tier), axis=1 )
        
        self.df['segment'] = self.df['combined_tier'].replace(self.segment_map, regex=True)
    
        rank_map = {
                    'Champion':1,
                   'Promising':2,
                   'Loyalist':3,
                   'Potential Loyalist':4,
                   'Unsteady':5,
                   'Needs Attention':6,
                    'At Risk':7,
                    'About to Lose':8
                   }
        self.df['segment_rank'] = self.df['segment'].apply(lambda x: rank_map[x])
        return self.df.copy()

In [ ]:

def generate_threshold(sbu,m, year, segment_map, order):
    #create 1 year rfm to generate 1-year threshold which will be applied to 2-year rfm
    end_date=str(year)+"-12-31"
    tr=[]
    sql_engine = sql_engine_server
    # sql_engine = sql.create_engine('mysql://mohammad:%s@147.139.173.222/?charset=utf8'% quote('P@ssw0rd2019!'))
    
    q=text(m['rfm_query'].replace('<end_period>',str(end_date)).replace('<n_year>',str(1)))
    print(q)
    df_rfm1 = pd.read_sql(q, sql_engine)
    
    rfm_base_model = rfm(df_rfm1.copy(),segment_map, order=order)
    print(rfm_base_model.threshold)
    
    
    df_tr = pd.DataFrame()
    df_tr['threshold'] = [rfm_base_model.threshold,]
    df_tr['SBU']= sbu
    df_tr['mapping']= q
    df_tr['period'] = end_date
    
    df_tr.to_sql(con=sql_engine, name='fact_rfm_threshold_sbu', schema='dev_db', if_exists='append',index=False)
    del df_rfm1
    del rfm_base_model

In [ ]:

#RFM SCHEMA 2023
order_list = ['mfr']

segment_map = { r'1[1-2]1': 'Champion',
            
            r'1[1-2]2': 'Promising',
            r'1[3-4][1-2]': 'Promising',
            
            r'1[1-4]3': 'Loyalist',
            r'2[1-2][1-3]': 'Loyalist',

            r'1[1-4]4': 'Potential Loyalist',
            r'2[1-2]4': 'Potential Loyalist',
            r'2[3-4][1-4]': 'Potential Loyalist',
 
            r'3[1-3][1-4]': 'Unsteady',
            
            r'34[1-4]': 'Needs Attention',
            
            r'4[1-3][1-4]': 'At Risk',
            r'44[1-2]': 'At Risk',

            r'44[1-4]': 'About to Lose'}

## iii. Execution

In [ ]:
#make sure temp helper table was ready

sql_engine = sql_engine_server
sql_engine.execute("""TRUNCATE TABLE dev_db.temp_rfm_market """)
sql_engine.execute("""TRUNCATE TABLE dev_db.temp_rfm_market_forgotten_users """)
sql_engine.dispose()

In [ ]:

# with end period only
# with helper
for i in all_sbu_mapping :
    
    # ORI
    # print(i)
    sbuname_ = list(i.keys())[0]
    print(sbuname_)
    
    prev_year=datetime.strptime(end_period,'%Y-%m-%d').year-1
    
    # for end_period in end_period_list:
    
    #new make helper
    sql_engine = sql_engine_server
    q=text(i[sbuname_]['helper_rfm'].replace('<end_period>',str(end_period)).replace('<n_year>', str(2)))
    print(q)
    
    df_rfm0 = pd.read_sql(q, sql_engine)
    df_rfm0.to_sql(con=sql_engine, name='temp_rfm_market', schema='dev_db',if_exists='append', index=False, chunksize=30000) ##
    sql_engine.dispose()
    del df_rfm0
    print('rfm market created')
    
    
    sql_engine = sql_engine_server
    q=text(i[sbuname_]['rfm_query'].replace('<end_period>',str(end_period)).replace('<n_year>', str(2)))
    print(q)
    df_rfm1 = pd.read_sql(q, sql_engine)

    
    for v in order_list:
    #get the data and storage it into df and upload it to server
        sql_engine = sql_engine_server
        df_check= pd.read_sql('select * from dev_db.fact_rfm_sbu where end_period= "%s" and sbu = "%s" limit 1' % (end_period, sbuname_), con=sql_engine)
        if len(df_check) < 1:
            df_tr=pd.read_sql('select threshold from dev_db.fact_rfm_threshold_sbu where sbu ="%s" and year(period)= %s  limit 1' % (sbuname_, prev_year), con=sql_engine)  
            if len(df_tr)<1:
                generate_threshold(sbuname_, i[sbuname_], prev_year, segment_map, order = v)
                df_tr=pd.read_sql('select threshold from dev_db.fact_rfm_threshold_sbu where sbu ="%s" and year(period)= %s  limit 1' % (sbuname_, prev_year), con=sql_engine)
            try:
                threshold = json.loads(df_tr.threshold.iloc[0].replace("'",'"'))
            except:
                _error_concept.append(sbuname_)
                continue
            
            if len(df_rfm1) > 0 :
                #initiate rfm instance
                rfm_static = rfm(df_rfm1.copy(),segment_map, order= v)
                #set threshold manually using the 1-year threshold
                rfm_static.set_threshold(threshold=threshold)
                #execute rfm
                df_rfm_static=rfm_static.fit() 
                df_rfm_static.reset_index(drop=True, inplace=True)
                #save rfm to db
                
                # sql_engine = sql_engine_server
                # sql_engine = sql.create_engine('mysql://'+username+':'+password+'@'+host+'/?charset=utf8mb4')
                
                sql_engine = sql_engine_server
                df_rfm_static.to_sql(con=sql_engine, name='fact_rfm_sbu', schema='dev_db',if_exists='append', index=False, chunksize=30000) ##
                sql_engine.dispose()
                print(v+' '+ sbuname_ +' '+end_period+' inserted')
                sql_engine.dispose()
                

                
        #check if forgotten_user for the designated SBU and end_period already exist. if not, create the forgotten_user
        # df_check2= pandas.read_sql('select * from gclub.forgotten_usersV2_concept where end_period= "%s" and concept = "%s" limit 1' % (end_period,i), con=sql_engine)
        sql_engine = sql_engine_server
        df_check2= pd.read_sql('select * from dev_db.fact_forgotten_user_sbu where end_period= "%s" and sbu = "%s" limit 1' % (end_period, sbuname_), con=sql_engine)
        
        if len(df_check2) < 1:
            # ORI
            # q=text(mapping[i]['forgotten_user_query'].replace('<end_period>',str(end_period)).replace('<count_current_rfm>',' '+str(df_rfm1.member_id.nunique())+' '))
            #new make helper
            sql_engine = sql_engine_server
            q=text(i[sbuname_]['helper_forgot'].replace('<end_period>',str(end_period)).replace('<n_year>', str(2)))
            print(q)
            
            df_rfm0 = pd.read_sql(q, sql_engine)
            df_rfm0.to_sql(con=sql_engine, name='temp_rfm_market_forgotten_users', schema='dev_db',if_exists='append', index=False, chunksize=30000) ##
            sql_engine.dispose()
            del df_rfm0
            print('rfm market forgotton created')
            
            sql_engine = sql_engine_server
            q=text(i[sbuname_]['forgotten_user_query'].replace('<end_period>',str(end_period)).replace('<count_current_rfm>',' '+str(df_rfm1.member_id.nunique())+' '))
            df_fu = pd.read_sql(q, con=sql_engine)
            sql_engine.dispose()
            
            sql_engine = sql_engine_server
            df_fu.to_sql(con=sql_engine, name='fact_forgotten_user_sbu', schema='dev_db',if_exists='append', index=False, chunksize=30000) ##
            sql_engine.dispose()
            
            print('forggooton user done')

        sql_engine = sql_engine_server
        sql_engine.execute("""TRUNCATE TABLE dev_db.temp_rfm_market """)
        sql_engine.execute("""TRUNCATE TABLE dev_db.temp_rfm_market_forgotten_users """)
        sql_engine.dispose()

## iv. Cleansing

In [ ]:
del df_rfm1, df_rfm_static, df_fu
gc.collect()

# END